In [1]:
/* What is the total amount each customer spent at the restaurant? */
select s.customer_id AS MEMBER_NAME, SUM(m.price) AS AMOUNT_SPENT
FROM sales AS s INNER JOIN menu AS m 
ON s.product_id = m.product_id
GROUP BY customer_id
ORDER BY customer_id;

(3 rows affected)

Total execution time: 00:00:00.141

MEMBER_NAME,AMOUNT_SPENT
A,76
B,74
C,36


In [12]:
/* How many days has each customer visited the restaurant? */
SELECT customer_id AS MEMBER_NAME, COUNT(DISTINCT order_date) AS NO_OF_DAYS_VISITED
FROM sales
GROUP BY customer_id;


(3 rows affected)

Total execution time: 00:00:00.016

MEMBER_NAME,NO_OF_DAYS_VISITED
A,4
B,6
C,2


In [16]:
/* What was the first item from the menu purchased by each customer? */
WITH cte_order AS (
    SELECT s.customer_id, m.product_name,
    ROW_NUMBER() OVER(
        PARTITION BY s.customer_id
        ORDER BY s.order_date,
        s.product_id
    ) AS first_purchase
    FROM sales AS s JOIN menu AS m
    ON s.product_id = m.product_id
)

SELECT * from cte_order WHERE first_purchase = 1;


(3 rows affected)

Total execution time: 00:00:00.028

customer_id,product_name,first_purchase
A,sushi,1
B,curry,1
C,ramen,1


In [12]:
/*What is the most purchased item on the menu and how many times was it purchased by all customers?*/
SELECT TOP 1 m.product_name, COUNT(s.product_id) AS PURCHASE_COUNT
FROM sales AS s INNER JOIN menu AS m
ON s.product_id = m.product_id
GROUP BY product_name
ORDER BY PURCHASE_COUNT DESC


(1 row affected)

Total execution time: 00:00:00.016

product_name,PURCHASE_COUNT
ramen,8


In [2]:
/*Which item was the most popular for each customer?*/
WITH popular_order_cte AS (
    SELECT s.customer_id, m.product_name, COUNT(s.product_id) AS order_count,
    ROW_NUMBER() OVER(
        PARTITION BY s.customer_id
        ORDER BY COUNT(s.customer_id) DESC
    ) AS order_rank
    FROM sales AS s JOIN menu AS m
    ON s.product_id = m.product_id
    GROUP BY s.customer_id, m.product_name
)

SELECT * from popular_order_cte where order_rank=1;

(3 rows affected)

Total execution time: 00:00:00.048

customer_id,product_name,order_count,order_rank
A,ramen,3,1
B,sushi,2,1
C,ramen,3,1


In [5]:
/*Which item was purchased first by the customer after they became a member?*/
WITH member_cte AS (
    SELECT s.customer_id, mem.join_date, s.order_date, s.product_id,
    ROW_NUMBER() OVER(
        PARTITION BY s.customer_id
        ORDER BY s.order_date) AS order_rank
    FROM sales AS s JOIN members AS mem
    ON s.customer_id = mem.customer_id
    WHERE s.order_date >= mem.join_date
    )
    SELECT s.customer_id, s.order_date, m.product_name
     from member_cte AS s JOIN menu AS m 
     ON s.product_id = m.product_id
     WHERE order_rank = 1;



(2 rows affected)

Total execution time: 00:00:00.037

customer_id,order_date,product_name
A,2021-01-07,curry
B,2021-01-11,sushi


In [10]:
/*Which item was purchased just before the customer became a member?*/
WITH member_cte AS (

    SELECT s.customer_id, mem.join_date, s.order_date, s.product_id,
    DENSE_RANK() OVER(
        PARTITION BY s.customer_id
        ORDER BY s.order_date DESC) AS order_rank
    FROM sales AS s JOIN members AS mem
    ON s.customer_id = mem.customer_id
    WHERE s.order_date < mem.join_date
    )
    SELECT s.customer_id, s.order_date, m.product_name 
    FROM member_cte s JOIN menu m 
    ON s.product_id = m.product_id
    WHERE order_rank=1;



(3 rows affected)

Total execution time: 00:00:00.023

customer_id,order_date,product_name
A,2021-01-01,sushi
A,2021-01-01,curry
B,2021-01-04,sushi


In [14]:
/*What is the total items and amount spent for each member before they became a member?*/
SELECT s.customer_id, COUNT(s.product_id) AS items_bought, SUM(m.price) AS total_amount
FROM sales AS s 
JOIN members AS mem
ON s.customer_id = mem.customer_id
JOIN menu AS m 
ON s.product_id = m.product_id
WHERE s.order_date < mem.join_date
GROUP BY s.customer_id;


(2 rows affected)

Total execution time: 00:00:00.029

customer_id,items_bought,total_amount
A,2,25
B,3,40


In [2]:
/*If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?*/
WITH points_cte AS(
    SELECT *,
    CASE
    WHEN product_id= 1
    THEN price * 20
    ELSE price * 10
    END AS points
    FROM menu
)
SELECT s.customer_id, SUM(p.points) AS total_points
FROM sales AS s JOIN points_cte AS p 
ON s.product_id = p.product_id
GROUP BY s.customer_id;


(3 rows affected)

Total execution time: 00:00:00.015

customer_id,total_points
A,860
B,940
C,360


In [11]:
/*In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?*/

/*Calculating validity of 2X POINTS PROGRAM*/

WITH valid_dates_cte AS
(
    SELECT *,
    DATEADD(DAY, 6, join_date) AS valid_date,
    EOMONTH('2021-01-31') AS last_date
    FROM members AS mem
)

SELECT v.customer_id, v.join_date, v.valid_date, v.last_date, s.order_date, m.product_name, m.price,
SUM(CASE
    WHEN s.order_date >= v.join_date AND s.order_date < v.valid_date THEN 2 * 10 * m.price 
    WHEN m.product_name = 'SUSHI' THEN 2 * 10 * m.price
    ELSE 10 * m.price 
    END) AS points 
FROM valid_dates_cte AS v JOIN
sales AS s ON v.customer_id = s.customer_id 
JOIN menu AS m 
ON s.product_id = m.product_id
WHERE s.order_date < v.last_date
GROUP BY v.customer_id, s.order_date, v.join_date, v.valid_date, v.last_date, m.product_name, m.price;

(10 rows affected)

Total execution time: 00:00:00.087

customer_id,join_date,valid_date,last_date,order_date,product_name,price,points
A,2021-01-07,2021-01-13,2021-01-31,2021-01-01,curry,15,150
A,2021-01-07,2021-01-13,2021-01-31,2021-01-01,sushi,10,200
A,2021-01-07,2021-01-13,2021-01-31,2021-01-07,curry,15,300
A,2021-01-07,2021-01-13,2021-01-31,2021-01-10,ramen,12,240
A,2021-01-07,2021-01-13,2021-01-31,2021-01-11,ramen,12,480
B,2021-01-09,2021-01-15,2021-01-31,2021-01-01,curry,15,150
B,2021-01-09,2021-01-15,2021-01-31,2021-01-02,curry,15,150
B,2021-01-09,2021-01-15,2021-01-31,2021-01-04,sushi,10,200
B,2021-01-09,2021-01-15,2021-01-31,2021-01-11,sushi,10,200
B,2021-01-09,2021-01-15,2021-01-31,2021-01-16,ramen,12,120


In [5]:
/*BONUS QUESTION ONE*/
SELECT s.customer_id, s.order_date, m.product_name, m.price,
CASE 
    WHEN mem.join_date > s.order_date THEN 'N'
    WHEN mem.join_date <= s.order_date THEN 'Y'
    ELSE 'N'
    END AS valid_member
FROM sales AS s LEFT JOIN menu AS m ON s.product_id = m.product_id 
LEFT JOIN members AS mem
ON s.customer_id = mem.customer_id;

(15 rows affected)

Total execution time: 00:00:00.028

customer_id,order_date,product_name,price,valid_member
A,2021-01-01,sushi,10,N
A,2021-01-01,curry,15,N
A,2021-01-07,curry,15,Y
A,2021-01-10,ramen,12,Y
A,2021-01-11,ramen,12,Y
A,2021-01-11,ramen,12,Y
B,2021-01-01,curry,15,N
B,2021-01-02,curry,15,N
B,2021-01-04,sushi,10,N
B,2021-01-11,sushi,10,Y


In [8]:
/*Bonus Question Two*/
WITH overall_rank_cte AS(
SELECT s.customer_id, s.order_date, m.product_name, m.price,
CASE 
    WHEN mem.join_date > s.order_date THEN 'N'
    WHEN mem.join_date <= s.order_date THEN 'Y'
    ELSE 'N'
    END AS valid_member
FROM sales AS s LEFT JOIN menu AS m ON s.product_id = m.product_id 
LEFT JOIN members AS mem
ON s.customer_id = mem.customer_id
)

SELECT *,
CASE
WHEN valid_member = 'N' THEN NULL
ELSE
RANK () OVER(PARTITION BY customer_id, valid_member
ORDER BY order_date) 
END AS member_ranking
FROM overall_rank_cte;


(15 rows affected)

Total execution time: 00:00:00.021

customer_id,order_date,product_name,price,valid_member,member_ranking
A,2021-01-01,sushi,10,N,NULL
A,2021-01-01,curry,15,N,NULL
A,2021-01-07,curry,15,Y,1
A,2021-01-10,ramen,12,Y,2
A,2021-01-11,ramen,12,Y,3
A,2021-01-11,ramen,12,Y,3
B,2021-01-01,curry,15,N,NULL
B,2021-01-02,curry,15,N,NULL
B,2021-01-04,sushi,10,N,NULL
B,2021-01-11,sushi,10,Y,1
